<a href="https://colab.research.google.com/github/venkatanadikatla/pytorch/blob/main/RNN_Building_Training_and_Evaluation_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip uninstall -y torchtext
!pip install torchtext==0.6.0

Found existing installation: torchtext 0.18.0
Uninstalling torchtext-0.18.0:
  Successfully uninstalled torchtext-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached n

In [1]:
import copy # this module provides functions to duplicate objects. It seems to be imported but not yet used in the code
import torch # The MAIN PyTorch package
from torch import nn # contains the essential modules for building NN in pytorch.
from torch import optim # Provides optimization algorithms, such as SGD, Adam, etc
import torchtext # A library for text processing that works well with pytorch (Currently a version 0.6.0 is being used in this code)
from torchtext import data # A module in torchtext used for data handling
from torchtext import datasets # provides datasets, including various NLP datasets.

TEXT = data.Field(sequential=True, batch_first=True, lower=True) # Sequential = True indicates that the data consists of sequences.
#Batch_first=True Ensure that batch dimenstion is the first dimension in the tensor. # lower=True Converts all the text to lowercase

LABEL = data.LabelField() # A subclass of Field specifically for handling labels in a classification task.

# load data splits
train_data, val_data, test_data = datasets.SST.splits(TEXT, LABEL) #datasets.SST.splits - Loads the Standford Sentiment Treebank(SST) dataset and splits the dataset

# build dictionary
# build_vocab: Creates a mapping from tokens(words) to indices. This is essential for converting text data into numerical form that can be used by NN.
TEXT.build_vocab(train_data) # Builds the vocabulary for the text field using the training data.
LABEL.build_vocab(train_data)# Builds the vocabulary for the label field using the training data.

# hyperparameters
vocab_size = len(TEXT.vocab) # the size of the vocabulary (number of unique tokens in the training data)
label_size = len(LABEL.vocab) # the number of unique labels (classes) in the traning data
padding_idx = TEXT.vocab.stoi['<pad>'] # The index used for padding sequences to the same length
embedding_dim = 128 # The size of the word embeddings (dense vector representation of words)
hidden_dim = 128 # Size of the hidden layers in the model

# build iterators
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size=32)

# Data.bucketiterator.splits - Creates iterators for the training, validation and test sets.


downloading trainDevTestTrees_PTB.zip


trainDevTestTrees_PTB.zip: 100%|██████████| 790k/790k [00:00<00:00, 2.26MB/s]


extracting


In [2]:
print(vars (train_data.examples[0]))

{'text': ['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'s", 'new', '``', 'conan', "''", 'and', 'that', 'he', "'s", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', ',', 'jean-claud', 'van', 'damme', 'or', 'steven', 'segal', '.'], 'label': 'positive'}


In [3]:
print(vars(val_data.examples[0]))

{'text': ['it', "'s", 'a', 'lovely', 'film', 'with', 'lovely', 'performances', 'by', 'buy', 'and', 'accorsi', '.'], 'label': 'positive'}


In [4]:
print(vars(test_data.examples[0]))

{'text': ['effective', 'but', 'too-tepid', 'biopic'], 'label': 'neutral'}


In [5]:
print(f'Num Train: {len(train_data)}')
print(f'Num Val: {len(val_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 8544
Num Val: 1101
Num Test: 2210


In [6]:
print(f'Vocabulary size: {len(TEXT.vocab)}')

Vocabulary size: 16581


In [7]:
print(f'Vocabulary size: {len(LABEL.vocab)}')

Vocabulary size: 3


In [8]:
print(LABEL.vocab.freqs)


Counter({'positive': 3610, 'negative': 3310, 'neutral': 1624})


In [9]:
print(TEXT.vocab.freqs.most_common(20))

[('.', 8024), ('the', 7303), (',', 7131), ('a', 5281), ('and', 4473), ('of', 4396), ('to', 3021), ('is', 2561), ("'s", 2544), ('it', 2422), ('that', 1954), ('in', 1888), ('as', 1296), ('but', 1172), ('film', 1162), ('with', 1139), ('for', 1023), ('this', 998), ('movie', 976), ('an', 972)]


In [10]:
print(TEXT.vocab.itos[:10])
#Tokens corresponding to the first 10 indices (0,1....9)

['<unk>', '<pad>', '.', 'the', ',', 'a', 'and', 'of', 'to', 'is']


In [11]:
class RNN (torch.nn.Module):
  def __init__(self, input_dim, embeddding_dim, hidden_dim, output_dim):
    super().__init__()
    self.embedding = torch.nn.Embedding(input_dim, embedding_dim, padding_idx = padding_idx)
    # self.rnn = torch.nn.RNN(embedding_dim,hidden_dim, nonlinearity='relu')
    self.lstm = torch.nn.LSTM(embedding_dim,hidden_dim, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, output_dim)


    def forward(self, text):
      #text dim = [sentence length, batchsize]
      embedded = self.embedding(text)
      #embedded dim = [sentence length, batchsize, embedding_dim]
      output, (hidden, cell) = self.lstm(embedded)
      #output dim = [sentence length, batchsize, hidden_dim]
      #hidden dim = [1, batch_size, hidden_dim]
      hidden = hidden.squeeze_(0)
      #hidden_dim = [batch_size, hidden_dim]

      return self.fc(hidden)



In [12]:
torch.manual_seed(42)
model = RNN(vocab_size,embedding_dim,hidden_dim,label_size)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()